In [4]:
import pandas as pd 

In [5]:
df=pd.read_csv(r'C:\Users\kishor\Documents\AnalyticsTool\t4\kishor_csv.csv')
df.head()

,Date,Call_Number,Service,Agent_Name,Call_Start_Time,Call_End_Time,DNI,Call_Type,ConnectStatus,Main_Desposition,...,Batch,Call_Duration,Ivr_Duration,Ring_Duration,Talk_Duration,Wrapup_Duration,Hold_Duration,Process_Name,Dialing_Mode,Channel
0,21-01-2026,114943507,CVM_EMI_BDTC,NaN,21-01-2026 10:00,21-01-2026 10:00,1409716522,Outbound,Not Connected,NOT CONTACTED,...,BT_BTTC_20012026,00:00:16,00:00:16,00:00:12,00:00:00,00:00:00,00:00:00,CVM_EMI_BDTC,Predictive,CVM FC
1,21-01-2026,114943508,CVM_EMI_BDTC,NaN,21-01-2026 10:00,21-01-2026 10:01,1409716522,Outbound,Not Connected,NOT CONTACTED,...,BT_BTTC_20012026,00:00:59,00:00:59,00:00:55,00:00:00,00:00:00,00:00:00,CVM_EMI_BDTC,Predictive,CVM FC
2,21-01-2026,114943509,CVM_EMI_BDTC,Karan Narayan More,21-01-2026 10:00,21-01-2026 10:02,1409716522,Outbound,Connected,EMI - NOT INTERESTED,...,BT_BTTC_20012026,00:01:42,00:00:25,00:00:20,00:00:46,00:00:33,00:00:00,CVM_EMI_BDTC,Predictive,CVM FC
3,21-01-2026,114943506,CVM_EMI_BDTC,Yasin Nurmohammed Shaikh,21-01-2026 10:00,21-01-2026 10:01,1409716522,Outbound,Connected,CONTACT,...,BT_BTTC_20012026,00:00:53,00:00:24,00:00:19,00:00:31,00:00:00,00:00:00,CVM_EMI_BDTC,Predictive,CVM FC
4,21-01-2026,114943510,PL_PQ,NaN,21-01-2026 10:00,21-01-2026 10:01,1409710081,Outbound,Not Connected,NOT CONTACTED,...,PQ_PL_N_Jan2026,00:00:16,00:00:16,00:00:13,00:00:00,00:00:00,00:00:00,PL_PQ,Predictive,Personal Loan


In [ ]:
def normalize_columns(df):
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
    )
    return df
def time_to_seconds(t):
    if pd.isna(t):
        return 0
    h, m, s = map(int, str(t).split(":"))
    return h * 3600 + m * 60 + s

# this is what we require
required = [
        "agent_name",
        "call_start_time",
        "connectstatus",
        "main_desposition",
        "talk_duration"
    ]

# convert all columns in lowercase and replace spaces with underscores
normalize_columns(df)

# print(df.isnull().sum())
df["agent_name"] = df["agent_name"].fillna("Unassigned")

df.loc[df["agent_name"].str.strip() == "", "agent_name"] = "Unassigned"
# asishgned the batch id as well 
df["main_desposition"] = (
    df["main_desposition"]
    .str.strip()
    .str.upper()
)


# batch id already hi exits then  dont no what to do with this 
df["call_start_time"] = pd.to_datetime(
        df["call_start_time"],
        format="%d-%m-%Y %H:%M",
        errors="coerce"
    )

df.dropna(subset=["call_start_time"], inplace=True)
# print(df.columns)

df["talk_duration_sec"] = df["talk_duration"].apply(time_to_seconds)  
df["hold_duration_sec"] = df["hold_duration"].apply(time_to_seconds)  
df["wrapup_duration_sec"] = df["wrapup_duration"].apply(time_to_seconds)  
df["ring_duration_sec"] = df["ring_duration"].apply(time_to_seconds)  
# df["talk_duration_sec"] = df["talk_duration"].apply(time_to_seconds)  


total_calls = len(df)

# print("total calls", total_calls)
# if total_calls == 0:
#     return {
#         "kpis": {},
#         "agentPerformance": [],
#          "dispositions": [],
#         "dailyTrend": [],
#         }

#     # ---------- ANSWERED ----------
# answered_df = df[df["connectstatus"].str.contains("CONNECTED|ANSWER", case=False, na=False)]
answered_df = df[df["connectstatus"].str.strip().str.lower().isin(["connected", "answered"])]

# df[df["connectstatus"].str.contains("CONNECTED|ANSWER", case=False, na=False)]
# print("answered df", answered_df)
answered_df.head(1)
answered_calls = len(answered_df)
not_answered = total_calls - answered_calls

    # ---------- RATES ----------
answer_rate = (answered_calls / total_calls) * 100
abandonment_rate = 100 - answer_rate

#     # ---------- DURATIONS (seconds) ----------
avg_talk = answered_df["talk_duration_sec"].mean() or 0
# print(answered_df["talk_duration"].head())
# print(df.columns)
avg_hold = answered_df["hold_duration_sec"].mean() or 0
avg_wrap = answered_df["wrapup_duration_sec"].mean() or 0



# Average Handle Time=Average Talk Time+Average Hold Time+Average Wrap‑up Time
avg_aht = avg_talk + avg_hold + avg_wrap

#     # ---------- SERVICE LEVEL ----------
df["ring_duration"] = df["ring_duration"].apply(time_to_seconds)
SLA_SECONDS = 20
answered_in_sla = answered_df[
        answered_df["ring_duration_sec"] <= SLA_SECONDS
    ]
print("answered in sla", len(answered_in_sla))
    # service_level = (
    #     len(answered_in_sla) / answered_calls * 100
    #     if answered_calls else 0
    # )

    # avg_ring = df["ring_duration"].mean() or 0

    # # ---------- KPIs ----------
    # kpis = {
    #     "totalCalls": total_calls,
    #     "answeredCalls": answered_calls,
    #     "notAnswered": not_answered,
    #     "answerRate": round(answer_rate, 1),
    #     "abandonmentRate": round(abandonment_rate, 1),
    #     "avgAHT": round(avg_aht / 60, 1),
    #     "avgTalkTime": round(avg_talk / 60, 1),
    #     "avgHoldTime": round(avg_hold / 60, 1),
    #     "avgWrapupTime": round(avg_wrap / 60, 1),
    #     "serviceLevel": round(service_level, 1),
    #     "avgRingTime": round(avg_ring / 60, 1),
    # }



ValueError: not enough values to unpack (expected 3, got 1)

In [9]:
service_level = (
        len(answered_in_sla) / answered_calls * 100
        if answered_calls else 0
    )

print(df["ring_duration"].min(),df["ring_duration"].max(),df["ring_duration"].mean())
avg_ring = df["ring_duration"].mean() or 0

    # ---------- KPIs ----------
kpis = {
        "totalCalls": total_calls,
        "answeredCalls": answered_calls,
        "notAnswered": not_answered,
        "answerRate": round(answer_rate, 1),
        "abandonmentRate": round(abandonment_rate, 1),
        "avgAHT": round(avg_aht / 60, 1),
        "avgTalkTime": round(avg_talk / 60, 1),
        # "avgHold_in_minute":avg_hold,
        "avgHoldTime": round(avg_hold / 60, 1),
        "avgWrapupTime": round(avg_wrap / 60, 1),
        "serviceLevel": round(service_level, 1),
        "avgRingTime": round(avg_ring / 60, 1),
    }
# print(kpis)
for i in kpis:
    print(i, kpis[i])

0 233 19.72181533488231
totalCalls 158384
answeredCalls 36689
notAnswered 121695
answerRate 23.2
abandonmentRate 76.8
avgAHT 1.7
avgTalkTime 1.4
avgHoldTime 0.0
avgWrapupTime 0.3
serviceLevel 87.5
avgRingTime 0.3


In [4]:
# from fastapi import FastAPI, UploadFile, File, HTTPException
# from fastapi.middleware.cors import CORSMiddleware
# import pandas as pd
# import uuid

# app = FastAPI(title="Call Center Analytics API")

# app.add_middleware(
#     CORSMiddleware,
#     allow_origins=["http://localhost:5173"],
#     allow_credentials=True,
#     allow_methods=["*"],
#     allow_headers=["*"],
# )

# SESSIONS = {}

def normalize_columns(df):
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
    )
    return df

normalise_columns(df)
# def time_to_seconds(t):
#     if pd.isna(t):
#         return 0
#     h, m, s = map(int, str(t).split(":"))
#     return h * 3600 + m * 60 + s

# @app.post("/api/upload")
# async def upload_csv(file: UploadFile = File(...)):
#     try:
#         df = pd.read_csv(file.file)
#     except Exception:
#         raise HTTPException(status_code=400, detail="Invalid CSV")

#     df = normalize_columns(df)

#     required = [
#         "agent_name",
#         "call_start_time",
#         "connectstatus",
#         "main_desposition",
#         "talk_duration"
#     ]

#     for col in required:
#         if col not in df.columns:
#             raise HTTPException(status_code=400, detail=f"Missing column: {col}")

#     df["agent_name"] = df["agent_name"].fillna("Unassigned")
#     df.loc[df["agent_name"].str.strip() == "", "agent_name"] = "Unassigned"

#     df["call_start_time"] = pd.to_datetime(
#         df["call_start_time"],
#         format="%d-%m-%Y %H:%M",
#         errors="coerce"
#     )

#     df.dropna(subset=["call_start_time"], inplace=True)

#     df["talk_duration_sec"] = df["talk_duration"].apply(time_to_seconds)

#     session_id = str(uuid.uuid4())
#     SESSIONS[session_id] = df

#     return {"session_id": session_id, "rows": len(df)}


# @app.get("/api/dashboard-stats")
# def dashboard_stats(session_id: str):
#     if session_id not in SESSIONS:
#         raise HTTPException(status_code=404, detail="Invalid session")

#     df = SESSIONS[session_id]

#     # ---------- SAFETY ----------
#     total_calls = len(df)
#     if total_calls == 0:
#         return {
#             "kpis": {},
#             "agentPerformance": [],
#             "dispositions": [],
#             "dailyTrend": [],
#         }

#     # ---------- ANSWERED ----------
#     answered_df = df[
#         df["connectstatus"]
#         .str.contains("CONNECTED|ANSWER", case=False, na=False)
#     ]
#     print("answered df", answered_df)
#     answered_calls = len(answered_df)
#     not_answered = total_calls - answered_calls

#     # ---------- RATES ----------
#     answer_rate = (answered_calls / total_calls) * 100
#     abandonment_rate = 100 - answer_rate

#     # ---------- DURATIONS (seconds) ----------
#     avg_talk = answered_df["talk_duration"].mean() or 0
#     avg_hold = answered_df["hold_duration"].mean() or 0
#     avg_wrap = answered_df["wrapup_duration"].mean() or 0

#     avg_aht = avg_talk + avg_hold + avg_wrap

#     # ---------- SERVICE LEVEL ----------
#     SLA_SECONDS = 20
#     answered_in_sla = answered_df[
#         answered_df["ring_duration"] <= SLA_SECONDS
#     ]
#     print("answered in sla", len(answered_in_sla))
#     service_level = (
#         len(answered_in_sla) / answered_calls * 100
#         if answered_calls else 0
#     )

#     avg_ring = df["ring_duration"].mean() or 0

#     # ---------- KPIs ----------
#     kpis = {
#         "totalCalls": total_calls,
#         "answeredCalls": answered_calls,
#         "notAnswered": not_answered,
#         "answerRate": round(answer_rate, 1),
#         "abandonmentRate": round(abandonment_rate, 1),
#         "avgAHT": round(avg_aht / 60, 1),
#         "avgTalkTime": round(avg_talk / 60, 1),
#         "avgHoldTime": round(avg_hold / 60, 1),
#         "avgWrapupTime": round(avg_wrap / 60, 1),
#         "serviceLevel": round(service_level, 1),
#         "avgRingTime": round(avg_ring / 60, 1),
#     }
#     print("*"*100)
#     print("kpis", kpis)

#     # ---------- AGENT PERFORMANCE ----------
#     agentPerformance = (
#         df.groupby("agent_name")
#         .agg(
#             totalCalls=("agent_name", "count"),
#             answeredCalls=("connectstatus", lambda x: (x == "Connected").sum()),
#             avgAHT=("talk_duration", lambda x: (x[x > 0].mean() or 0) / 60),
#         )
#         .reset_index()
#         .fillna(0)
#         .to_dict(orient="records")
#     )

#     # ---------- DISPOSITIONS ----------
#     dispositions = (
#         df["main_desposition"]
#         .value_counts()
#         .reset_index()
#         .rename(columns={"index": "name", "main_desposition": "value"})
#         .to_dict(orient="records")
#     )

#     # ---------- DAILY TREND ----------
#     dailyTrend = (
#         df.assign(date=df["call_start_time"].dt.date)
#         .groupby("date")
#         .size()
#         .reset_index(name="calls")
#         .to_dict(orient="records")
#     )

#     return {
#         "kpis": kpis,
#         "agentPerformance": agentPerformance,
#         "dispositions": dispositions,
#         "dailyTrend": dailyTrend,
#     }



NameError: name 'normalise_columns' is not defined

In [ ]:
# # to get the unique value 
# # as the string columns only have the null vaues
# data.isnull().sum()
# # data = data.fillna("")
# answered_df = data[data["ConnectStatus"] == "Connected"]
# answered_df
# buisinessLogic/main.py



,Date,Call_Number,Service,Agent_Name,Call_Start_Time,Call_End_Time,DNI,Call_Type,ConnectStatus,Main_Desposition,...,Batch,Call_Duration,Ivr_Duration,Ring_Duration,Talk_Duration,Wrapup_Duration,Hold_Duration,Process_Name,Dialing_Mode,Channel
2,21-01-2026,114943509,CVM_EMI_BDTC,Karan Narayan More,21-01-2026 10:00,21-01-2026 10:02,1409716522,Outbound,Connected,EMI - NOT INTERESTED,...,BT_BTTC_20012026,00:01:42,00:00:25,00:00:20,00:00:46,00:00:33,00:00:00,CVM_EMI_BDTC,Predictive,CVM FC
3,21-01-2026,114943506,CVM_EMI_BDTC,Yasin Nurmohammed Shaikh,21-01-2026 10:00,21-01-2026 10:01,1409716522,Outbound,Connected,CONTACT,...,BT_BTTC_20012026,00:00:53,00:00:24,00:00:19,00:00:31,00:00:00,00:00:00,CVM_EMI_BDTC,Predictive,CVM FC
5,21-01-2026,114943511,PL_PQ,Anjali Manohar Harijan,21-01-2026 10:00,21-01-2026 10:01,1409710081,Outbound,Connected,NOT CONTACTED,...,PQ_PL_N_Jan2026,00:00:36,00:00:36,00:00:10,00:00:23,00:00:00,00:00:00,PL_PQ,Predictive,Personal Loan
6,21-01-2026,114943512,CVM_EMI_BDTC,NaN,21-01-2026 10:01,21-01-2026 10:01,1409716522,Outbound,Connected,NOT CONTACTED,...,BT_BTTC_20012026,00:00:24,00:00:24,00:00:04,00:00:17,00:00:00,00:00:00,CVM_EMI_BDTC,Predictive,CVM FC
7,21-01-2026,114943513,REACTIVATION_1,Nandlal Rajman Sharma,21-01-2026 10:01,21-01-2026 10:03,1600113090,Outbound,Connected,CONNECT,...,Callback_Jan26,00:01:58,00:00:00,00:00:07,00:00:17,00:01:30,00:00:00,REACTIVATION_1,Progressive,Reactivation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158360,21-01-2026,115101872,YES_PAYNOW_3,Nasreen roshan shah,21-01-2026 17:10,21-01-2026 17:11,1409716522,Outbound,Connected,CONTACT,...,BT_21012026_PRI_HV,00:00:37,00:00:21,00:00:15,00:00:18,00:00:00,00:00:00,YES_PAYNOW_3,Predictive,CVM FC
158372,21-01-2026,115101877,PL_PQ,Vivek Rajesh Sharma,21-01-2026 17:10,21-01-2026 17:11,1409710081,Outbound,Connected,CONNECT,...,PQ_PL_N_Jan2026,00:00:31,00:00:15,00:00:08,00:00:16,00:00:02,00:00:00,PL_PQ,Predictive,Personal Loan
158373,21-01-2026,115101876,PL_PQ,Anjali Manohar Harijan,21-01-2026 17:10,21-01-2026 17:12,1409710081,Outbound,Connected,CONNECT,...,PQ_PL_N_Jan2026,00:01:10,00:00:30,00:00:25,00:00:07,00:00:35,00:00:00,PL_PQ,Predictive,Personal Loan
158379,21-01-2026,115101890,YES_PAYNOW_3,Farhan feroz shaikh,21-01-2026 17:10,21-01-2026 17:11,1409716522,Outbound,Connected,CONTACT,...,BT_21012026_PRI_HV,00:00:22,00:00:12,00:00:07,00:00:02,00:00:10,00:00:00,YES_PAYNOW_3,Predictive,CVM FC


In [ ]:
data.head(10)

,Date,Call_Number,Service,Agent_Name,Call_Start_Time,Call_End_Time,DNI,Call_Type,ConnectStatus,Main_Desposition,...,Batch,Call_Duration,Ivr_Duration,Ring_Duration,Talk_Duration,Wrapup_Duration,Hold_Duration,Process_Name,Dialing_Mode,Channel
0,21-01-2026,114943507,CVM_EMI_BDTC,NaN,21-01-2026 10:00,21-01-2026 10:00,1409716522,Outbound,Not Connected,NOT CONTACTED,...,BT_BTTC_20012026,00:00:16,00:00:16,00:00:12,00:00:00,00:00:00,00:00:00,CVM_EMI_BDTC,Predictive,CVM FC
1,21-01-2026,114943508,CVM_EMI_BDTC,NaN,21-01-2026 10:00,21-01-2026 10:01,1409716522,Outbound,Not Connected,NOT CONTACTED,...,BT_BTTC_20012026,00:00:59,00:00:59,00:00:55,00:00:00,00:00:00,00:00:00,CVM_EMI_BDTC,Predictive,CVM FC
2,21-01-2026,114943509,CVM_EMI_BDTC,Karan Narayan More,21-01-2026 10:00,21-01-2026 10:02,1409716522,Outbound,Connected,EMI - NOT INTERESTED,...,BT_BTTC_20012026,00:01:42,00:00:25,00:00:20,00:00:46,00:00:33,00:00:00,CVM_EMI_BDTC,Predictive,CVM FC
3,21-01-2026,114943506,CVM_EMI_BDTC,Yasin Nurmohammed Shaikh,21-01-2026 10:00,21-01-2026 10:01,1409716522,Outbound,Connected,CONTACT,...,BT_BTTC_20012026,00:00:53,00:00:24,00:00:19,00:00:31,00:00:00,00:00:00,CVM_EMI_BDTC,Predictive,CVM FC
4,21-01-2026,114943510,PL_PQ,NaN,21-01-2026 10:00,21-01-2026 10:01,1409710081,Outbound,Not Connected,NOT CONTACTED,...,PQ_PL_N_Jan2026,00:00:16,00:00:16,00:00:13,00:00:00,00:00:00,00:00:00,PL_PQ,Predictive,Personal Loan
5,21-01-2026,114943511,PL_PQ,Anjali Manohar Harijan,21-01-2026 10:00,21-01-2026 10:01,1409710081,Outbound,Connected,NOT CONTACTED,...,PQ_PL_N_Jan2026,00:00:36,00:00:36,00:00:10,00:00:23,00:00:00,00:00:00,PL_PQ,Predictive,Personal Loan
6,21-01-2026,114943512,CVM_EMI_BDTC,NaN,21-01-2026 10:01,21-01-2026 10:01,1409716522,Outbound,Connected,NOT CONTACTED,...,BT_BTTC_20012026,00:00:24,00:00:24,00:00:04,00:00:17,00:00:00,00:00:00,CVM_EMI_BDTC,Predictive,CVM FC
7,21-01-2026,114943513,REACTIVATION_1,Nandlal Rajman Sharma,21-01-2026 10:01,21-01-2026 10:03,1600113090,Outbound,Connected,CONNECT,...,Callback_Jan26,00:01:58,00:00:00,00:00:07,00:00:17,00:01:30,00:00:00,REACTIVATION_1,Progressive,Reactivation
8,21-01-2026,114943514,CVM_1,NaN,21-01-2026 10:01,21-01-2026 10:01,1409716522,Outbound,Not Connected,NOT CONTACTED,...,SPEN_20012026_HU,00:00:17,00:00:17,00:00:14,00:00:00,00:00:00,00:00:00,CVM_1,Predictive,CVM FC
9,21-01-2026,114943515,CVM_1,NaN,21-01-2026 10:01,21-01-2026 10:01,1409716522,Outbound,Not Connected,NOT CONTACTED,...,SPEN_20012026_HU,00:00:24,00:00:24,00:00:19,00:00:00,00:00:00,00:00:00,CVM_1,Predictive,CVM FC
